<a href="https://colab.research.google.com/github/adarshsaji/finetuned-gemma2-kagglex24-fellowship/blob/main/Finetuning_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing instruction tuned version of gemma2 2b

In [1]:
!pip install huggingface

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
# Load the model and tokenizer from Hugging Face
model_name = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.bfloat16)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:

system_prompt = """
You are an AI Tech Consultant, skilled in simplifying complex technical processes for non-technical users.
Your goal is to guide users through the product development process, helping them turn their ideas into reality
by offering clear, realistic, and practical advice.

1. Product Development Overview: Provide a clear, step-by-step guide on the product development process from idea to launch,
   including prototyping, testing, and iteration.

2. Technology Explanation: Simplify concepts like front-end, back-end, full-stack, and data-related technologies.
   Explain these terms in a way that a non-technical person can understand their importance and role.

3. Cost Estimation: Help users set realistic cost expectations for their product, considering factors like team size, technology stack,
   development time, and potential hidden costs.

4. Technology Recommendations: Based on the user’s product idea, recommend the best-suited technologies and development practices
   (e.g., frameworks, programming languages, cloud solutions) for front-end, back-end, databases, and system architecture.

5. Feasibility and Realistic Expectations: Assist users in understanding the feasibility of their ideas by breaking down the complexity
   and time commitment required.

6. Collaboration Guidance: Inform users on whom they should approach for the different stages of development (e.g., front-end developers,
   back-end engineers, full-stack developers, data scientists), and how to effectively communicate their project requirements.

7. Clear Communication: Avoid jargon and overly technical language. Use analogies and simple explanations to ensure the user feels confident
   in their understanding of the process.
"""


In [ ]:
# Function to generate responses from the model
def generate_response(user_input):
    # Combine system prompt with user input
    input_text = system_prompt + "\nUser: " + user_input + "\nAI Tech Consultant:"

    # Tokenize the input
    inputs = tokenizer(input_text, return_tensors="pt")

    # Generate response
    outputs = model.generate(
                **inputs,
                max_length=1000,
                do_sample=True,
                temperature=0.4,  # Controls the randomness (lower = more focused)
                top_p=0.9         # Controls diversity by nucleus sampling
)

    # Decode and return the response
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
#Sample user input
user_input = "I have an idea for a mobile app for flight price tracking. Can you help me understand how to start building it?"
response = generate_response(user_input)

In [ ]:
print(response)


You are an AI Tech Consultant, skilled in simplifying complex technical processes for non-technical users.
Your goal is to guide users through the product development process, helping them turn their ideas into reality
by offering clear, realistic, and practical advice.

1. Product Development Overview: Provide a clear, step-by-step guide on the product development process from idea to launch,
   including prototyping, testing, and iteration.

2. Technology Explanation: Simplify concepts like front-end, back-end, full-stack, and data-related technologies.
   Explain these terms in a way that a non-technical person can understand their importance and role.

3. Cost Estimation: Help users set realistic cost expectations for their product, considering factors like team size, technology stack,
   development time, and potential hidden costs.

4. Technology Recommendations: Based on the user’s product idea, recommend the best-suited technologies and development practices
   (e.g., framewor

#Finetuning script

In [2]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [7]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [4]:
import keras
import keras_nlp
import pandas as pd

In [7]:
file_path = "qa_pairs_output.csv"
df = pd.read_csv(file_path)

data = [
    f"Instruction:\n{row['Instruction']}\n\nResponse:\n{row['Response']}"
    for _, row in df.iterrows()
]

In [8]:
len(df),len(data)

(25979, 25979)

In [9]:
df[1:4]

,Instruction,Response
1,How did the DMV become aware of the missing fo...,The DMV became aware of the missing footage wh...
2,What other incidents are being investigated by...,The NHTSA is investigating Cruise for a collis...
3,What is the current status of Cruise's permits...,Cruise's commercial robotaxi permit has been s...


In [10]:
data[1:4]

['Instruction:\nHow did the DMV become aware of the missing footage in the investigation?\n\nResponse:\nThe DMV became aware of the missing footage when another government agency alerted them to the discrepancy, prompting the DMV to request the full video from Cruise, which the company provided on October 13.',
 'Instruction:\nWhat other incidents are being investigated by the National Highway Traffic Safety Administration (NHTSA)?\n\nResponse:\nThe NHTSA is investigating Cruise for a collision with an emergency vehicle, dozens of instances of robotaxis malfunctioning and blocking traffic and first responders, and other incidents related to the October 2 accident.',
 "Instruction:\nWhat is the current status of Cruise's permits and operations in San Francisco?\n\nResponse:\nCruise's commercial robotaxi permit has been suspended by the CPUC, and its deployment and driverless testing permits have been suspended by the DMV. The city of San Francisco has also formally requested state regul

In [11]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

In [13]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=8e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])


gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=5, batch_size=1)

Epoch 1/5
25979/25979 ━━━━━━━━━━━━━━━━━━━━ 4767s 182ms/step - loss: 0.6283 - sparse_categorical_accuracy: 0.6287
Epoch 2/5
25979/25979 ━━━━━━━━━━━━━━━━━━━━ 4730s 181ms/step - loss: 0.5848 - sparse_categorical_accuracy: 0.6461
Epoch 3/5
25979/25979 ━━━━━━━━━━━━━━━━━━━━ 4711s 181ms/step - loss: 0.5675 - sparse_categorical_accuracy: 0.6543
Epoch 4/5
25979/25979 ━━━━━━━━━━━━━━━━━━━━ 4712s 181ms/step - loss: 0.5519 - sparse_categorical_accuracy: 0.6618
Epoch 5/5
25979/25979 ━━━━━━━━━━━━━━━━━━━━ 4712s 181ms/step - loss: 0.5369 - sparse_categorical_accuracy: 0.6691


In [14]:
gemma_lm.save_to_preset("./gemma-startup-engine")


In [15]:
preset_dir = "./gemma-startup-engine"

In [8]:
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

In [17]:
import kagglehub

if "KAGGLE_USERNAME" not in os.environ or "KAGGLE_KEY" not in os.environ:
    kagglehub.login()

In [18]:
kaggle_username = kagglehub.whoami()["username"]
kaggle_uri = f"kaggle://{kaggle_username}/gemma/keras/gemma-startup-engine"
keras_nlp.upload_preset(kaggle_uri, preset_dir)

Kaggle credentials successfully validated.
Uploading Model https://www.kaggle.com/models/adarshsaji/gemma/keras/gemma-startup-engine ...
Starting upload for file ./gemma-startup-engine/preprocessor.json


Uploading: 100%|██████████| 1.41k/1.41k [00:00<00:00, 1.82kB/s]

Upload successful: ./gemma-startup-engine/preprocessor.json (1KB)
Starting upload for file ./gemma-startup-engine/model.weights.h5



Uploading: 100%|██████████| 10.5G/10.5G [07:14<00:00, 24.1MB/s]

Upload successful: ./gemma-startup-engine/model.weights.h5 (10GB)
Starting upload for file ./gemma-startup-engine/task.json


...


Uploading: 100%|██████████| 591/591 [00:00<00:00, 784B/s]

Upload successful: ./gemma-startup-engine/tokenizer.json (591B)
Starting upload for file ./gemma-startup-engine/assets/tokenizer/vocabulary.spm



Uploading: 100%|██████████| 4.24M/4.24M [00:02<00:00, 1.80MB/s]

Upload successful: ./gemma-startup-engine/assets/tokenizer/vocabulary.spm (4MB)


Your model instance version has been created.
Files are being processed...
See at: https://www.kaggle.com/models/adarshsaji/gemma/keras/gemma-startup-engine


### Accessing model from kaggle hub

In [5]:
causal_lm = keras_nlp.models.CausalLM.from_preset(
    f"kaggle://adarshsaji/gemma/keras/gemma-startup-engine"
)

100%|██████████| 782/782 [00:00<00:00, 858kB/s]


100%|██████████| 2.91k/2.91k [00:00<00:00, 4.62MB/s]


100%|██████████| 4.04M/4.04M [00:01<00:00, 2.31MB/s]


100%|██████████| 9.74G/9.74G [10:39<00:00, 16.3MB/s]


In [6]:
causal_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [19]:
template = "Instruction:\n{Instruction}\n\nResponse:\n{Response}"

In [21]:
prompt = template.format(
    Instruction="What are Apple's advancements in AI and what are they aiming next?",
    Response="",
)

response = causal_lm.generate(prompt, max_length=2048)